In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as sts
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats.distributions import chi2
import seaborn as sns
import yfinance as yf
sns.set()

In [2]:
tickers=['^NSEBANK','^NSEI','^BSESN']
data = pd.DataFrame()
for ticker in tickers:
    data[ticker]=yf.download(ticker,start='2010-01-01',end='2025-7-7')['Close']

C:\Users\swana\AppData\Local\Temp\ipykernel_16816\26573317.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[ticker]=yf.download(ticker,start='2010-01-01',end='2025-7-7')['Close']
[*********************100%***********************]  1 of 1 completed
C:\Users\swana\AppData\Local\Temp\ipykernel_16816\26573317.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[ticker]=yf.download(ticker,start='2010-01-01',end='2025-7-7')['Close']
[*********************100%***********************]  1 of 1 completed
C:\Users\swana\AppData\Local\Temp\ipykernel_16816\26573317.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[ticker]=yf.download(ticker,start='2010-01-01',end='2025-7-7')['Close']
[*********************100%***********************]  1 of 1 completed


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3532 entries, 2010-01-04 to 2025-07-04
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ^NSEBANK  3532 non-null   float64
 1   ^NSEI     3530 non-null   float64
 2   ^BSESN    3529 non-null   float64
dtypes: float64(3)
memory usage: 110.4 KB


In [4]:
data.head()

,^NSEBANK,^NSEI,^BSESN
Date,,,
2010-01-04,9112.244141,5232.200195,17558.730469
2010-01-05,9192.043945,5277.899902,17686.240234
2010-01-06,9222.892578,5281.799805,17701.130859
2010-01-07,9192.843750,5263.100098,17615.720703
2010-01-08,9160.593750,5244.750000,17540.289062


In [5]:
data.isna().sum()

^NSEBANK    0
^NSEI       2
^BSESN      3
dtype: int64

In [6]:
## Dropping Null Values
data.dropna(inplace=True)

In [7]:
data.isna().sum()

^NSEBANK    0
^NSEI       0
^BSESN      0
dtype: int64

In [8]:
data.head()

,^NSEBANK,^NSEI,^BSESN
Date,,,
2010-01-04,9112.244141,5232.200195,17558.730469
2010-01-05,9192.043945,5277.899902,17686.240234
2010-01-06,9222.892578,5281.799805,17701.130859
2010-01-07,9192.843750,5263.100098,17615.720703
2010-01-08,9160.593750,5244.750000,17540.289062


In [9]:
data=data.asfreq('B')

In [10]:
data.head()

,^NSEBANK,^NSEI,^BSESN
Date,,,
2010-01-04,9112.244141,5232.200195,17558.730469
2010-01-05,9192.043945,5277.899902,17686.240234
2010-01-06,9222.892578,5281.799805,17701.130859
2010-01-07,9192.843750,5263.100098,17615.720703
2010-01-08,9160.593750,5244.750000,17540.289062


In [11]:
data.isna().sum()

^NSEBANK    518
^NSEI       518
^BSESN      518
dtype: int64

In [12]:
data.fillna(method='ffill',inplace=True)

C:\Users\swana\AppData\Local\Temp\ipykernel_16816\286314875.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill',inplace=True)


In [13]:
data.isna().sum()

^NSEBANK    0
^NSEI       0
^BSESN      0
dtype: int64

In [14]:
## Simplyfying the Datasets
data['Market_value']=data['^NSEBANK']

In [15]:
data.head()

,^NSEBANK,^NSEI,^BSESN,Market_value
Date,,,,
2010-01-04,9112.244141,5232.200195,17558.730469,9112.244141
2010-01-05,9192.043945,5277.899902,17686.240234,9192.043945
2010-01-06,9222.892578,5281.799805,17701.130859,9222.892578
2010-01-07,9192.843750,5263.100098,17615.720703,9192.843750
2010-01-08,9160.593750,5244.750000,17540.289062,9160.593750


In [17]:
del data['^NSEBANK']

In [ ]:
data.head()

,Market_value
Date,
2010-01-04,9112.244141
2010-01-05,9192.043945
2010-01-06,9222.892578
2010-01-07,9192.843750
2010-01-08,9160.593750


In [ ]:
data.describe()

,Market_value
count,4045.000000
mean,25076.327577
std,13321.497160
min,7798.458984
25%,12424.555664
50%,22264.642578
75%,35123.988281
max,57459.449219


In [ ]:
## Spliting the Date
size=int(len(data)*0.8)
train=data.iloc[:size]
test=data.iloc[size:]

In [18]:
train.tail()

NameError: name 'train' is not defined

In [ ]:
## copying train data to df for simplefying
df=train.copy()

In [21]:
test.head()

,Market_value
Date,
2022-05-31,35486.984375
2022-06-01,35620.386719
2022-06-02,35613.234375
2022-06-03,35274.640625
2022-06-06,35309.789062
